In [1]:
from IPython.display import display
import yaml
import lib.database_module as db
import lib.encoding_module as enc
import lib.wiki_module as wiki
import pandas as pd

### Mine: Select document vectors for all pages from database

In [ ]:
con, cur = db.connect_to_postgres()

In [ ]:
page_vectors = db.select_all_page_vectors()
indices = [tup[0] for tup in page_vectors]
vectors = [tup[1] for tup in page_vectors]

In [15]:
page_vec = pd.DataFrame(vectors, index=indices)
page_vec.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
4341789,0.092992,-0.040985,0.006952,0.000243,0.001681,0.004999,0.003430,0.003523,-0.005129,-0.001911,...,-0.021003,-0.039528,0.027716,0.021033,-0.034768,-0.022294,0.031537,-0.009710,-0.010331,0.025944
48201744,0.021301,0.027559,-0.003917,-0.010877,0.013055,0.000934,0.000128,0.010317,0.001444,-0.009792,...,-0.055040,0.000110,0.057817,0.005156,0.058477,0.008807,0.043527,-0.011693,0.064738,0.051296
2514975,0.124983,0.153025,0.026264,-0.257191,-0.210183,0.074847,0.003862,-0.009666,-0.065705,-0.280972,...,-0.022703,0.015894,-0.002476,0.004992,-0.006653,-0.017394,-0.008525,0.017188,-0.001596,0.000120
35135520,0.191357,-0.039003,-0.006018,0.006389,-0.001106,-0.004748,0.015829,0.005612,0.011846,0.028248,...,0.000578,-0.004204,0.030916,0.010484,-0.012454,0.026050,0.038744,0.015332,-0.008053,0.009506
27303975,0.034292,0.039499,-0.006095,-0.015156,0.026112,-0.039749,-0.081440,-0.036546,0.005076,0.018881,...,0.030056,-0.026965,-0.002426,0.042282,0.024082,0.004498,-0.009517,-0.005315,-0.003784,-0.010202


### Mine: Select category ids corresponding to pages from database

In [23]:
page_cats = db.execute_sql_statement('''SELECT * FROM page_cate''')

OK
Connected to server joshuacook.me.


In [25]:
page_cats = pd.DataFrame(page_cats, columns=['page_id','category_id'])

In [28]:
joined_df = pd.merge(page_vec.reset_index(), page_cats, left_on='index', right_on='page_id')
joined_df.head()

,index,0,1,2,3,4,5,6,7,8,...,492,493,494,495,496,497,498,499,page_id,category_id
0,4341789,0.092992,-0.040985,0.006952,0.000243,0.001681,0.004999,0.003430,0.003523,-0.005129,...,0.027716,0.021033,-0.034768,-0.022294,0.031537,-0.009710,-0.010331,0.025944,4341789,695196
1,48201744,0.021301,0.027559,-0.003917,-0.010877,0.013055,0.000934,0.000128,0.010317,0.001444,...,0.057817,0.005156,0.058477,0.008807,0.043527,-0.011693,0.064738,0.051296,48201744,716903
2,2514975,0.124983,0.153025,0.026264,-0.257191,-0.210183,0.074847,0.003862,-0.009666,-0.065705,...,-0.002476,0.004992,-0.006653,-0.017394,-0.008525,0.017188,-0.001596,0.000120,2514975,1489690
3,35135520,0.191357,-0.039003,-0.006018,0.006389,-0.001106,-0.004748,0.015829,0.005612,0.011846,...,0.030916,0.010484,-0.012454,0.026050,0.038744,0.015332,-0.008053,0.009506,35135520,695196
4,27303975,0.034292,0.039499,-0.006095,-0.015156,0.026112,-0.039749,-0.081440,-0.036546,0.005076,...,-0.002426,0.042282,0.024082,0.004498,-0.009517,-0.005315,-0.003784,-0.010202,27303975,34955640


### Refine: Create a data dictionary with training and testing sets

Begin development using local dummy data.

In [81]:
import pandas as pd

In [32]:
X = joined_df.drop(['category_id','index','page_id'], axis=1)
y = joined_df['category_id']

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [36]:
data_dict = {'X': X,
             'y': y,
             'X_train': X_train,
             'y_train': y_train,
             'X_test': X_test,
             'y_test': y_test 
}

### Model: Fit, Score, and Tune a multi-label classification model

In [37]:
import numpy as np

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [38]:
knn = KNeighborsClassifier()

In [47]:
y_train = np.ravel(y_train)

In [39]:
y_train.shape

(1788,)

In [44]:
X_train.shape

(1788, 500)

In [45]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [46]:
knn.score

<bound method KNeighborsClassifier.score of KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')>

In [43]:
knn.predict

<bound method KNeighborsClassifier.predict of KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')>

In [47]:
from sklearn.naive_bayes import GaussianNB

In [48]:
this_nc = GaussianNB()

In [49]:
this_nc.fit(data_dict['X_train'], data_dict['y_train'])

GaussianNB(priors=None)

In [50]:
this_nc_train_score = this_nc.score(data_dict['X_train'], data_dict['y_train'])
this_nc_test_score = this_nc.score(data_dict['X_test'], data_dict['y_test'])

In [51]:
print ('Train: {} Test: {}' .format(this_nc_train_score, this_nc_test_score))

Train: 0.878635346756 Test: 0.790268456376


In [52]:
from sklearn import tree

In [53]:
clf = tree.DecisionTreeClassifier()

In [54]:
clf.fit(data_dict['X_train'], data_dict['y_train'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [55]:
clf_train_score = clf.score(data_dict['X_train'], data_dict['y_train'])
clf_test_score = clf.score(data_dict['X_test'], data_dict['y_test'])

In [56]:
print ('Train: {} Test: {}' .format(clf_train_score, clf_test_score))

Train: 0.961968680089 Test: 0.827181208054


### Present: Pickle tuned model for later use

In [57]:
from sklearn.externals import joblib

In [61]:
joblib.dump(this_nc, 'my_pickle_model.pickle')

['my_pickle_model.pickle']